## 一、 概括 ：此文档主要LR相关知识和答案以及部分代码。  
### 1、代码函数简介
#### code在logistic_mini文件内 建议pycharm打开  
#### 其中run.py是入口

#### Non_negative_cons 函数解问题1；order_cons 是解问题2 ；logistic_reg 验证LR全局最优

#### 数据使用的是 sklearn.load_breast_cancer  

#### logistic_mini.py是类似sklearn的LogisticRegression类，在fi函数新加了constraints 参数用于处理 LR引入constraints的优化问题；


### 2、问题2
Question 2

We are all very familiar with Logistic Regression and its optimization solution to minimize its cost function. But in reality, we may need some constraints. The following are two special constraints, please give: a) optimization solution; b) code of the solution process; c) sample code (you can use the sklearn data set or generate your own code)

1) Non-negative constraint: All coefficients of LR are required to be non-negative;

2) Order preserving constraint: The coefficient of LR is required to satisfy a1 >= a2 >= a3 ……

One more question: If there is no constraint, is LR a global optimization algorithm or a local one? why? When constraints 1) or 2) are imposed, global or local? why?

logistic regression with weights constraints(non-nagetive,desceding order)

### 3、问题分析：问题是定位为 在一定约束下，求解逻辑回归最优解  
**小问题1** ： 参数非负，即参数都是正的  
**小问题2** :  参数是有序的，满足 a1>=a2>=a3...  
**小问题3** ：全局优化和局部优化



### 4、解题总思路：
定位目标函数，选择优化方法，借助scipy,numpy，sklearn 使用方法 L-BFGS-B TNC trust-constr等


# 二、关于LR的基础知识
## 1、目标函数
棒极了！现在，我们需要编写代价函数来评估结果。
代价函数：
$J\left( \theta  \right)=\frac{1}{m}\sum\limits_{i=1}^{m}{[-{{y}^{(i)}}\log \left( {{h}_{\theta }}\left( {{x}^{(i)}} \right) \right)-\left( 1-{{y}^{(i)}} \right)\log \left( 1-{{h}_{\theta }}\left( {{x}^{(i)}} \right) \right)]}$  
本文使用的是 sklearn内的y={-1,1}  

<img src="loss_fun.png" style="zoom:50%"/>

## gradient 

* 转化为向量化计算： $\frac{1}{m} X^T( Sigmoid(X\theta) - y )$  

$$\frac{\partial J\left( \theta  \right)}{\partial {{\theta }_{j}}}=\frac{1}{m}\sum\limits_{i=1}^{m}{({{h}_{\theta }}\left( {{x}^{(i)}} \right)-{{y}^{(i)}})x_{_{j}}^{(i)}}$$


# 三、解题
# 问题1: Non-negative constraint

## 1、optimization solution 
针对LR 在constraints下的优化，使用了scipy.optimize.minimize 模块，method主要使用了 L-BFGS-B，TNC优化方法，  

**L-BFGS-B**  
L-BFGS算法是牛顿法的改进  

阶段分为牛顿法，BFGS，L-BFGS  
牛顿法的本质是泰勒级数的二阶展开  
BFGS:解决牛顿法求逆矩阵的困难，是通过迭代逼近逆矩阵的拟牛顿法  
L-BFGS(L指limited memory)
为了解决算法每次存储D矩阵，需要大内存的问题，只保存BFGS迭代计算中的后几次迭代  

**TNC**  

Minimize a function with variables subject to bounds, using gradient information in a truncated Newton algorithm

### bounds 
a1>=0,a2>=0,a3>=0....  
上线边界即   
  lb_ = np.zeros((X.shape[1]+1))  
  ub_ = np.full(X.shape[1]+1, np.inf)
## 2、sample code

In [1]:
def Non_negative_cons():
    X, y = load_breast_cancer(return_X_y=True)
    # sklearn y = {-1,1}
    y[y == 0] = -1
    lb_ = np.zeros((X.shape[1] + 1))

    ub_ = np.full(X.shape[1] + 1, np.inf)

    bounds = Bounds(lb_, ub_)
    print('lb_ {}'.format(lb_.shape))
    print('ub_ {}'.format(ub_.shape))

    ############
    ####  method 可选 L-BFGS-B and TNC
    ####  solver='lxw' 调用 _fix_lxw 执行minimize
    ####  max_iter 迭代次数
    #### fix_initial 是否更改初始值
    #### bounds 设置 constraints 边界，问题1 即 a1>=0,a2>=2,a3>=3....

    #   return param
    #   max_iter ,penalty（none 即没用正则）
    #   score 分类准确率
    #   coef_ coefficient
    #
    ############
    method = "L-BFGS-B"

    clf = LogisticRegression(solver="lxw", penalty="none", max_iter=150)
    # fix_initial 初值赋值更改，验证参数收敛是否一致
    #     clf = LogisticRegression(solver="lxw", penalty="none",max_iter=60,fix_initial=True)

    clf.fit(X, y, method=method, bounds=bounds)
    print('===================== {} ==========='.format(method))
    print('max_iter : {}'.format(clf.max_iter))
    print('penalty : {}'.format(clf.penalty))

    print('score : {}'.format(clf.score(X, y)))
    print('coef_  : {}'.format(clf.coef_))
    print('res: {}  optimizer : {}'.format(method, clf.res.success))

    # L-BFGS-B 迭代 50、60次，参数相同，有最优解，
    # == == == == == == == == == == = L - BFGS - B == == == == == =
    # max_iter: 50
    # penalty: none
    # score: 0.6274165202108963
    # coef_: [[0.         0.         0.         0.         0.00889884 0.
    #          0.         0.         0.0174621  0.01259587 0.         0.23684945
    #          0.         0.         0.00180309 0.         0.         0.
    #          0.00418088 0.00033521 0.         0.         0.         0.
    #          0.00669678 0.         0.         0.         0.00518067 0.00472869]]
    # res: L - BFGS - B
    # optimizer: True

    # ===================== L-BFGS-B ===========
    # max_iter : 60
    # penalty : none
    # score : 0.6274165202108963
    # coef_  : [[0.         0.         0.         0.         0.         0.
    #   0.         0.         0.         0.0020697  0.         0.24097333
    #   0.         0.         0.00400276 0.         0.         0.
    #   0.00099565 0.01133466 0.         0.         0.         0.
    #   0.         0.         0.         0.         0.         0.        ]]
    # res: L-BFGS-B  optimizer : True

    # 加入 l2的结果
    # == == == == == == == == == == = L - BFGS - B == == == == == =
    # max_iter: 50
    # penalty: l2
    # score: 0.6274165202108963
    # coef_: [[0.         0.         0.         0.         0.00888393 0.
    #          0.         0.         0.01743283 0.01257477 0.         0.23645251
    #          0.         0.         0.00180007 0.         0.         0.
    #          0.00417387 0.00033464 0.         0.         0.         0.
    #          0.00668555 0.         0.         0.         0.00517197 0.00472076]]
    # res: L - BFGS - B
    # optimizer: True

    # ===================== TNC ===========
    # max_iter : 40
    # penalty : none
    # score : 0.6274165202108963
    # coef_  : [[ 0.          0.          0.          0.          0.          0.
    #    0.          0.          0.          1.1488147   0.          0.
    #    0.          0.         45.21328606  0.          0.          0.
    #    0.          0.          0.          0.          0.          0.
    #    0.          0.          0.          0.          0.          0.        ]]
    # res: TNC  optimizer : False

    #####  =========== 迭代50次以后优化器退出，为true，coef_稳定全局解
    # ===================== TNC ===========
    # max_iter : 50
    # penalty : none
    # score : 0.6274165202108963
    # coef_  : [[ 0.         0.         0.         0.         0.         0.
    #    0.         0.         0.         0.         0.         0.
    #    0.         0.        49.0536372  0.         0.         0.
    #    0.         0.         0.         0.         0.         0.
    #    0.         0.         0.         0.         0.         0.       ]]
    # res: TNC  optimizer : True

    # ===================== TNC ===========
    # max_iter : 60
    # penalty : none
    # score : 0.6274165202108963
    # coef_  : [[ 0.         0.         0.         0.         0.         0.
    #    0.         0.         0.         0.         0.         0.
    #    0.         0.        49.0536372  0.         0.         0.
    #    0.         0.         0.         0.         0.         0.
    #    0.         0.         0.         0.         0.         0.       ]]
    # res: TNC  optimizer : True

    ##### ====== 加入L2正则 迭代70次 收敛
    # ===================== TNC ===========
    # max_iter : 70
    # penalty : l2
    # score : 0.6274165202108963
    # coef_  : [[0.         0.         0.         0.         0.         0.
    #   0.         0.         0.         0.02211936 0.         0.0303164
    #   0.         0.         0.05256518 0.         0.         0.
    #   0.0070915  0.         0.         0.         0.         0.
    #   0.         0.         0.         0.         0.         0.        ]]
    # res: TNC  optimizer : True

    ### 更改赋初值，coef_收敛相同值，迭代次数增加到150，LR 加入 Non-negative constraint 使用TNC 有全局最优解
    # ===================== TNC ===========
    # max_iter : 150
    # penalty : none
    # score : 0.6274165202108963
    # coef_  : [[ 0.          0.          0.          0.          0.          0.
    #    0.          0.          0.          0.          0.          0.
    #    0.          0.         49.05363767  0.          0.          0.
    #    0.          0.          0.          0.          0.          0.
    #    0.          0.          0.          0.          0.          0.        ]]
    # res: TNC  optimizer : True


# 问题2：Order preserving constraint 
## 1、optimization solution 
针对LR 在constraints下的优化，使用了scipy.optimize.minimize 模块，method主要使用了 trust-constr 优化方法
### trust-constr   
<img src="./trust_region.png" style="zoom:50%"/> 
具体可参考
https://epubs.siam.org/doi/abs/10.1137/0724076

### constraints
Order preserving constraint 即 a1>=a2>=a3>=a4...
使用scipy中的LinearConstrains 构建 constraints  
<img src="./math1.png" style="zoom:50%"/>  
Order preserving constraint 对应公式 即：  
<img src="./order_constraints.png" style="zoom:20%"/>    

## 2、sample code


In [ ]:
def order_cons():

    X, y = load_breast_cancer(return_X_y=True)

    lb = np.zeros((X.shape[1]))
    ub = np.full(X.shape[1], np.inf)

    # print('lb {}'.format(lb.shape))
    # print('un {}'.format(ub.shape))
    #
    A = np.zeros((X.shape[1], X.shape[1] + 1))
    for i in range(X.shape[1]):
        A[i, i:i + 2] = np.array([1, -1])

    # print('A {} {}'.format(A.shape, A[-1, -1]))

    constraints = LinearConstraint(A, lb, ub)

    clf = LogisticRegression(solver="lxw", penalty="none",max_iter=225)
    #      fix_initial 初值赋值更改，验证参数收敛是否一致
    # clf = LogisticRegression(solver="lxw", penalty="none",max_iter=225,fix_initial=True)
    ############
    ####  method trust-constr
    ####  solver='lxw' 调用 _fix_lxw 执行minimize
    ####  max_iter 迭代次数
    #### fix_initial 是否更改初始值
    #### onstraints 为 即 a1>=a2>=a3 ....

    #   return param
    #   max_iter ,penalty（none 即没用正则）
    #   score 分类准确率
    #   coef_ coefficient
    #
    ############
    method = "trust-constr"

    clf.fit(X, y, constraints=constraints, method=method)

    print('===================== {} ==========='.format(method))
    print('max_iter : {}'.format(clf.max_iter))
    print('penalty : {}'.format(clf.penalty))

    print('score : {}'.format(clf.score(X, y)))
    print('coef_  : {}'.format(clf.coef_))
    print('res: {}  optimizer : {}'.format(method, clf.res.success))

    # == == == == == == == == == == = trust - constr == == == == == =
    # max_iter: 225
    # penalty: none
    # score: 0.961335676625659
    # coef_: [[3.80424902 - 0.02360999 - 0.02360999 - 0.02360999 - 0.02360999 - 0.02360999
    #          - 0.02360999 - 0.02360999 - 0.02360999 - 0.02360999 - 0.02360999 - 0.02360999
    #          - 0.02360999 - 0.02360999 - 0.02360999 - 0.02360999 - 0.02360999 - 0.02360999
    #          - 0.02360999 - 0.02360999 - 0.02360999 - 0.02360999 - 0.02360999 - 0.02360999
    #          - 2.25030929 - 2.25030933 - 2.25030944 - 7.35546295 - 7.35546298 - 7.35546301]]
    # res: trust - constr
    # optimizer: True

    ## 更改初值 参数与上个初值的 参数一致，目标函数全局最优解
    # == == == == == == == == == == = trust - constr == == == == == =
    # max_iter: 225
    # penalty: none
    # score: 0.961335676625659
    # coef_: [[3.80424876 - 0.02360998 - 0.02360998 - 0.02360998 - 0.02360998 - 0.02360998
    #          - 0.02360998 - 0.02360998 - 0.02360998 - 0.02360998 - 0.02360999 - 0.02360999
    #          - 0.02360999 - 0.02360999 - 0.02360999 - 0.02360999 - 0.02360999 - 0.02360999
    #          - 0.02360999 - 0.02360999 - 0.02360999 - 0.02360999 - 0.02360999 - 0.02360999
    #          - 2.25030933 - 2.25030948 - 2.25030978 - 7.35546156 - 7.35546174 - 7.35546193]]
    # res: trust - constr
    # optimizer: True

    ## 引入 正则 l2
    # == == == == == == == == == == = trust - constr == == == == == =
    # max_iter: 225
    # penalty: l2
    # score: 0.9490333919156415
    # coef_: [[2.04013969 - 0.00463975 - 0.00463975 - 0.00463975 - 0.0221412 - 0.02214125
    #          - 0.02214127 - 0.02214127 - 0.02214127 - 0.02214128 - 0.02214128 - 0.02214128
    #          - 0.02214129 - 0.02214129 - 0.02214129 - 0.02214129 - 0.02214129 - 0.02214129
    #          - 0.02214129 - 0.02214129 - 0.02214129 - 0.02214129 - 0.02214129 - 0.02214129
    #          - 0.48456143 - 1.484345 - 1.48434501 - 1.48434502 - 1.48434502 - 1.48434503]]
    # res: trust - constr
    # optimizer: True

##  问题3：One more question
1、如果没有约束，LR is a global optimization algorithm！ 因为逻辑回归是凸函数，logistic_reg 函数可运行验证  
2、LR with Non-negative constraint 是全局最优，基于优化算法 L-BFGS-B，通过更改迭代次数和更改初值，参数收敛一致  
3、LR with preserving constraint 是全局最优，基于优化算法 trust_region，通过更改迭代次数和更改初值，参数收敛一致.且trust_region是总体收敛的
